In [5]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();
Pkg.add("ForwardDiff")

  Activating environment at `~/all_about_robotics/julia_optimisation_course/tutorial/mpc_control_examples/Project.toml`
Precompiling project...
  ✓ CodecBzip2
  ✓ IntelOpenMP_jll
  ✓ DualNumbers
  ✓ OSQP_jll
  ✓ libblastrampoline_jll
  ✓ EarCut_jll
  ✓ Rmath_jll
  ✓ MKL_jll
  ✓ Rmath
  ✓ PreallocationTools
  ✓ NLSolversBase
  ✓ HypergeometricFunctions
  ✓ SparseDiffTools
  ✓ StatsFuns
  ✓ FFTW
  ✓ LineSearches
  ✓ RobotDynamics
  ✓ NLsolve
  ✓ GeometryBasics
  ✓ DSP
  ✓ RobotZoo
  ✓ Distributions
  ✓ MeshCat
  ✓ LoopVectorization
  ✓ DiffEqBase
  ✓ ExponentialUtilities
  ✓ TriangularSolve
  ✓ SciMLNLSolve
  ✓ DiffEqCallbacks
  ✓ SimpleNonlinearSolve
  ✓ ControlSystemsBase
  ✓ RecursiveFactorization
  ✓ MathOptInterface
  ✓ OSQP
  ✓ LinearSolve
  ✓ NonlinearSolve
  ✓ OrdinaryDiffEq
  ✓ DelayDiffEq
  ✓ ControlSystems
  39 dependencies successfully precompiled in 186 seconds (189 already precompiled)
   Resolving package versions...
    Updating `~/all_about_robotics/julia_optimisation_co

In [4]:
using LinearAlgebra
using PyPlot
using SparseArrays
using ForwardDiff
using ControlSystems
using OSQP


LoadError: ArgumentError: Package FowardDiff not found in current path:
- Run `import Pkg; Pkg.add("FowardDiff")` to install the FowardDiff package.


In [ ]:
#Model parameters
g = 9.81 #m/s^2
m = 1.0 #kg 
ℓ = 0.3 #meters
J = 0.2*m*ℓ*ℓ

#Thrust limits
umin = [0.2*m*g; 0.2*m*g]
umax = [0.6*m*g; 0.6*m*g]

h = 0.05 #time step (20 Hz)


In [ ]:
#Planar Quadrotor Dynamics
function quad_dynamics(x,u)
    θ = x[3]
    
    ẍ = (1/m)*(u[1] + u[2])*sin(θ)
    ÿ = (1/m)*(u[1] + u[2])*cos(θ) - g
    θ̈ = (1/J)*(ℓ/2)*(u[2] - u[1])
    
    return [x[4:6]; ẍ; ÿ; θ̈]
end

In [ ]:
function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

In [ ]:
#Linearized dynamics for hovering
x_hover = zeros(6)
u_hover = [0.5*m*g; 0.5*m*g]
A = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,u_hover),x_hover);
B = ForwardDiff.jacobian(u->quad_dynamics_rk4(x_hover,u),u_hover);
quad_dynamics_rk4(x_hover, u_hover)

In [ ]:
Nx = 6     # number of state
Nu = 2     # number of controls
Tfinal = 10.0 # final time
Nt = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(Nt-1), step=h));

In [ ]:
# Cost weights
Q = Array(1.0*I(Nx));
# Cost weights
Q = Array(1.0*I(Nx));
R = Array(.01*I(Nu));
Qn = Array(1.0*I(Nx));R = Array(.01*I(Nu));
Qn = Array(1.0*I(Nx));

In [ ]:
#Cost function
function cost(xhist,uhist)
    cost = 0.5*xhist[:,end]'*Qn*xhist[:,end]
    for k = 1:(size(xhist,2)-1)
        cost = cost + 0.5*xhist[:,k]'*Q*xhist[:,k] + 0.5*(uhist[k]'*R*uhist[k])[1]
    end
    return cost
end

In [ ]:
#LQR Hover Controller

K = dlqr(A,B,Q,R)

function lqr_controller(t,x,K,xref)
    
    return u_hover - K*(x-xref)
end